In [1]:
!pip install pandarallel
!pip install pyahocorasick
!pip install --upgrade pip
!pip install spacy==3.0.*

  Created wheel for pandarallel: filename=pandarallel-1.5.2-py3-none-any.whl size=18384 sha256=58e1ebbc2930ef872a18d5a73ef3c9ce750c1462481bb1dda6947656caf64a1d
  Stored in directory: /root/.cache/pip/wheels/b5/6d/51/9ece2eaf007ea3f7fb0ce053c5773b2eb0d308887da3af12c1
Successfully built pandarallel
     |████████████████████████████████| 321 kB 863 kB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.1-cp37-cp37m-linux_x86_64.whl size=102845 sha256=a3e4d24b8846f841fadb655311e2b2edbbac56a52bf47d1b059f075ce2985a98
  Stored in directory: /root/.cache/pip/wheels/fe/ea/e6/38b0d734be6936b783e916a0d8d670313fb1b2f74c5889d4fe
Successfully built pyahocorasick
     |████████████████████████████████| 12.8 MB 288 kB/s 
     |████████████████████████████████| 456 kB 5.4 MB/s 
     |████████████████████████████████| 1.1 MB 5.8 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue

In [2]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)

from pandarallel import pandarallel
pandarallel.initialize()

import spacy
spacy.require_gpu()
from spacy.training import Example
from spacy.util import minibatch
import random

import ahocorasick

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
df = pd.read_csv("../input/scl-2021-ds/train.csv")
df.set_index("id", inplace=True)
df['POI'] = np.nan
df['street'] = np.nan

def extract_entities(row):
    extracted = row['POI/street'].split("/")
    
    if len(extracted) == 2:
        poi, street = extracted
        if poi.strip() != '':
            row['POI'] = poi
        
        if street.strip() != '':
            row['street'] = street
        
    return row

df = df.parallel_apply(extract_entities, axis=1)
nlp = spacy.blank('id')  # create blank Language class

In [4]:
from copy import deepcopy

def _build_aho(words):
    aho = ahocorasick.Automaton()
    for idx, key in enumerate(words):
        aho.add_word(key, (idx, key))

    return aho

def format_data(text, poi, street):
    entities = []
    _text = deepcopy(text)

    if isinstance(poi, str):
        aho = _build_aho([poi])
        aho.make_automaton()
        latest_char_idx = 0
        
        for end, (_, word) in aho.iter(_text):
            start = end - len(word) + 1
            if start < latest_char_idx:
                continue

            entities.append((start, end + 1, 'POI'))
            _text = _text.replace(word, " " * len(word))
            latest_char_idx = end + 1
        
    if isinstance(street, str):
        aho = _build_aho([street])
        aho.make_automaton()
        latest_char_idx = 0

        for end, (_, word) in aho.iter(_text):
            start = end - len(word) + 1
            if start < latest_char_idx:
                continue

            entities.append((start, end + 1, 'STREET'))
            latest_char_idx = end + 1
    
    return Example.from_dict(nlp.make_doc(text), {"entities": entities})

In [5]:
print("Preparing Spacy examples...")

examples = []
for idx in df.index:
    try:
        row = df.loc[idx]
        example = format_data(row['raw_address'], row['POI'], row['street'])
        examples.append(example)
    except Exception as e:
        print(idx)
        print("-" * 50)
        print(e)
        break

Preparing Spacy examples...


In [6]:
def train_spacy(nlp, examples, iterations):
    
    TRAIN_DATA = examples
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.add_pipe('ner', last=True)

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print("Starting iteration " + str(itn))
            random.shuffle(examples)
            batches = minibatch(examples, 1000)
            losses = {}
            for batch in batches:
                nlp.update(
                    batch,
                    drop=0.2,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print(losses)
    return nlp

In [7]:
train = examples

In [8]:
%%time
ner_nlp = train_spacy(nlp, train, 20)

[2021-03-15 11:55:18,485] [INFO] Created vocabulary
[2021-03-15 11:55:18,487] [INFO] Finished initializing nlp object


Starting iteration 0
{'ner': 323734.03137283004}
Starting iteration 1
{'ner': 170926.6240479609}
Starting iteration 2
{'ner': 143802.65496775782}
Starting iteration 3
{'ner': 129507.55024031132}
Starting iteration 4
{'ner': 121077.29372811098}
Starting iteration 5
{'ner': 111915.66885191426}
Starting iteration 6
{'ner': 106873.12344156954}
Starting iteration 7
{'ner': 103272.73400572392}
Starting iteration 8
{'ner': 97765.99141909205}
Starting iteration 9
{'ner': 96079.5418501288}
Starting iteration 10
{'ner': 92310.75271290338}
Starting iteration 11
{'ner': 90768.40105642738}
Starting iteration 12
{'ner': 88095.13012283432}
Starting iteration 13
{'ner': 86582.26926741986}
Starting iteration 14
{'ner': 84348.90447655541}
Starting iteration 15
{'ner': 82923.51859248859}
Starting iteration 16
{'ner': 79696.39765542251}
Starting iteration 17
{'ner': 80578.29333920995}
Starting iteration 18
{'ner': 79580.0670703959}
Starting iteration 19
{'ner': 77001.15006978271}
CPU times: user 2h 27min 

In [9]:
for idx, row in df.iloc[100:110].iterrows():
    print(f"address: {row['raw_address']}")
    print(f"expected poi: {row['POI']}")
    print(f"expected street: {row['street']}")
    print()
    
    doc = ner_nlp(row['raw_address'])
    for ent in doc.ents:
        print(ent.text, "-", ent.label_)

    print("-" * 50)

address: kedai tenun jep senn, kota bumi, kebon melati
expected poi: kedai tenun jepara sennaart
expected street: kota bumi

kota bumi - STREET
--------------------------------------------------
address: wadungasri dalam waru raya wad asri, 24 sidoarjo
expected poi: dalam waru
expected street: raya wad asri

waru raya wad asri - STREET
--------------------------------------------------
address: bulusan tim barat iii, no 35 3 tembalang
expected poi: nan
expected street: tim barat iii

tim barat iii - STREET
--------------------------------------------------
address: bakti jaya bukit perm vii 8 15315 setu
expected poi: nan
expected street: bukit perm vii

bukit perm vii - STREET
--------------------------------------------------
address: jl terusan buah batu no 185. samping indomaret. bandung.
expected poi: samping indomaret
expected street: jl terusan buah batu

jl terusan buah batu - STREET
--------------------------------------------------
address: setia indah, jati, no 12
expected po

In [10]:
df_test = pd.read_csv("../input/scl-2021-ds/test.csv")
df_test.set_index("id", inplace=True)

In [11]:
submission = []
for idx, row in df_test.iloc[:].iterrows():
    doc = ner_nlp(row['raw_address'])
    tmp = {'id': idx}
    for ent in doc.ents:
        tmp[ent.label_] = ent.text
    submission.append(tmp)
    


In [12]:
submission = pd.DataFrame(submission)
submission['POI'] = submission['POI'].replace(np.nan, '')
submission['STREET'] = submission['STREET'].replace(np.nan, '')
submission['POI/street'] = submission['POI'] + '/' + submission['STREET'] 

In [13]:
pd.DataFrame({'id':submission['id'],'POI/street':submission['POI/street']}).to_csv('submission.csv', header=True, index=False)